In [1]:
import pandas as pd
import bs4 as bs
import urllib.request
from tqdm import tqdm
import numpy as np
import json

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cd /content/drive/MyDrive/news-title-bias-master/notebooks

/content/drive/MyDrive/news-title-bias-master/notebooks


# load story metadata

In [20]:
url_root = 'https://www.allsides.com/'
url_page = url_root + 'story/admin?page='
tab_num = 63

# loop over pages
# there are about 100 pages as of 12/29/2020, so we set this number to be well above that
dfs = []
for tab_num in tqdm(range(tab_num,90)):
    # read in the source
    try:
        source = urllib.request.urlopen(url_page + str(tab_num))
        sp = bs.BeautifulSoup(source, 'lxml')
        table = sp.table
        df = pd.read_html(str(table), encoding='utf-8', header=0)[0] # read table with no links

        # get links
        links = []
        for tag in table.find_all('a'):
            if tag.has_attr('href'):
                link = tag.get('href')
                if '/story' in link:
                    links.append(link)
            else:
                print(f'error! missing a link for {link}')
        df['url_story'] = links
        dfs.append(df)
    except:
        print(f'stopped after {tab_num} pages')
        break
        
df = pd.concat(dfs)
df.to_pickle('../data/df_links.pkl')

df.to_csv('data.csv')

100%|██████████| 27/27 [00:46<00:00,  1.73s/it]


In [21]:
# add info from link
def get_info_from_url_story(url_story, t):
    '''add info rom url on a a story
    '''
    story = urllib.request.urlopen(url_story)
    sp_story = bs.BeautifulSoup(story, 'html.parser')

    
    # extract info from specific story page
    story_triplet_info = {}
    
    # loop over left, center, and right stories
    try:
        for div in sp_story.find_all('div', {'class': 'news-title'})[:3]:
            title = div.a.contents[0]
            url = div.a.get('href')
            topic = t
            #news_source = div.parent.find_all('div', {'class': 'news-source'})[0].contents[1]
            leaning = div.parent.find_all('div', {'class': 'bias-image'})[0].img.get('title').replace("Political News Media Bias Rating: ", '')

            #news_text = '\n'.join([s.contents[0] for s in 
            #         div.parent.find_all('div', {'class': 'news-body'})[0].contents
            #         if 'Tag' in str(type(s))])
            prefix = leaning.lower().replace('lean ', '') + '_story_'
            story_info = {
                f'{prefix}title': title,
                #f'{prefix}url': url,
                #f'{prefix}source': news_source,
                f'{prefix}leaning': leaning,
                f'{prefix}topic': t,
                #f'{prefix}text': news_text
            }
            story_triplet_info = {**story_triplet_info, **story_info}
            #print(story_triplet_info)
    except:
        return {}
    return story_triplet_info

def get_stories(df):
    '''Add list for all stories
    '''
    story_triplet_list = []
    for i in tqdm(range(df.shape[0])):
        url_story = url_root + df.iloc[i]['url_story']
        topic = str(df.iloc[i]['Topics'])
        story_triplet_info = get_info_from_url_story(url_story, topic)
        story_triplet_list.append(story_triplet_info)
    df_stories = pd.DataFrame.from_dict(story_triplet_list)
    for k in df_stories:
        df_stories[k] = df_stories[k].astype(str)
    return df_stories
    
df_stories = get_stories(df)

100%|██████████| 1350/1350 [21:51<00:00,  1.03it/s]


In [22]:
df_stories = df_stories[[k for k in df_stories.keys()
        if not 'mixed' in k and not 'not rated' in k]]

# drop rows with any nans
df_stories = df_stories.replace('nan', np.nan)
df_stories = df_stories.dropna()

In [23]:
df_stories.to_csv('headlines.csv')

In [15]:
print(len(df_stories.columns))

9
